In [26]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\KIIT\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\KIIT\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\KIIT\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [27]:
filename = 'amazon_in_urls.txt'
file = open('amazon_in_urls.txt')
LIST=re.split("\r\n|\n",file.read())
file.close()
# for i in LIST:
#     print(i.split('~')[-1])
#     url.append(i.split('~')[-1])
base_url = LIST[0].split('~')[-1]
product_name = str(LIST[1])
filename.split(".")

['amazon_in_urls', 'txt']

In [28]:
reviewlist = []
domains = [".com", ".co.uk", ".ca", ".in", ".co.mx", ".com.au", ".co.jp", ".sg", ".nl", ".de", ".fr", ".it", ".es", ".com.br"]
# domains = [".in"]

In [29]:
def get_soup(url):
    r = requests.get('http://localhost:8050/render.html', params={'url': url, 'wait': 2})
    soup = BeautifulSoup(r.text, 'html.parser')
    return soup

In [30]:
def get_reviews(soup):
    reviews = soup.find_all('div', {'data-hook': 'review'})
    try:
        for item in reviews:
            product_title = soup.title.text.strip()
            product_title = product_title.split("Intel", 1)[-1].strip()
            product_title = "Intel " + product_title

            review_date = item.find('span', {'data-hook': 'review-date'}).text.strip()
            review_date = ' '.join(review_date.split()[-3:])

            rating_text = item.find('i', {'data-hook': 'review-star-rating'}).text.strip()
            rating_value = re.search(r'(\d+(\.\d+)?)', rating_text).group(1)
            rating = float(rating_value)

            review = {
                'Product': product_title,
                'Rating': rating,
                'Date': review_date,
                'Review': item.find('span', {'data-hook': 'review-body'}).text.strip(),
            }
            reviewlist.append(review)
    except Exception as e:
        print(f"An error occurred: {e}")

In [31]:
for y in domains:
    for x in range(1,11):
        url = re.sub(r'www\.amazon\.[a-z\.]+', f'www.amazon{y}', base_url)
        url = f'{url}&pageNumber={x}'
        soup = get_soup(url)
        print(f'Getting page: {x} from {y}')
        initial_length = len(reviewlist)
        get_reviews(soup)
        print(len(reviewlist))
        print(url)
        if len(reviewlist) == initial_length:
            break
        if not soup.find('li', {'class': 'a-disabled a-last'}):
            pass
        else:
            break

Getting page: 1 .in
10
https://www.amazon.in/Intel%C2%AE-CoreTM-i5-10400-Processor-Cache/product-reviews/B086MN38Q2/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=1


In [40]:
custom_words = [
    "against", "above", "below", "out", "under", "again",
    "no", "nor", "not", "don't", "should", "didn't",
    "does", "doesn't", "hadn't", "hasn't", "haven't",
    "isn't", "mustn", "mustn't", "should", "shouldn't",
    "was", "wasn't", "weren", "weren't", "won't", "wouldn't",
    "very", "couldn't"
]

stop_words = set(stopwords.words('english'))
for word in custom_words:
    if word in stop_words:
        stop_words.remove(word)
lemmatizer = WordNetLemmatizer()

for i in range(len(reviewlist)):
    review_dict = reviewlist[i]
    review = review_dict['Review']
    
    # Remove non-alphabetic characters
    review = re.sub(r'[^A-Za-z\s]', '', review)
    
    # Convert to lowercase
    review = review.lower()
    
    # Tokenize and remove stopwords
    words = word_tokenize(review)
    filtered_words = [word for word in words if word not in stop_words]
    
    # Lemmatize
    lemmatized_review = ' '.join([lemmatizer.lemmatize(word) for word in filtered_words])
    
    review_dict['Review'] = lemmatized_review

{'with', 'or', 'both', "should've", 'in', 'has', 'being', 'as', 'its', 'yourselves', 'do', 'have', 'on', 'if', 'hasn', 'when', 't', 'and', 'shouldn', 'herself', 'by', 'are', 'd', 'they', 'off', 'some', 'through', "needn't", 'so', 'were', 'haven', 'once', 'him', 'but', 'm', 'at', 'wouldn', 'itself', 'further', 'to', "that'll", 'any', 'yours', 'here', 'why', "it's", 'mightn', 'own', 'just', "she's", 'then', 'now', 'doesn', 'where', 'too', "shan't", "you're", "aren't", 'a', 'ourselves', 'be', 'didn', 'don', 'had', 'myself', 'my', 'her', 'which', 'of', 'over', 'into', 'such', 'themselves', 'what', 'having', 'between', 'than', 'after', 'doing', 'those', 'that', 'isn', 'them', 'his', 'only', 'during', 'it', 'because', 'needn', 'other', 'ma', 'our', 'down', 'how', 'ours', 'there', 'won', 'about', 'am', 'few', 'their', 's', 'll', 'we', 'these', 've', 'will', 'he', 'the', 're', 'been', 'did', 'himself', 'i', 'before', 'more', 'all', 'hers', 'up', 'wasn', 'ain', "you'd", 'whom', 'this', 'shan', 

In [34]:
df = pd.DataFrame(reviewlist)
df.to_csv(f'../data/amazon/{product_name}.csv', index=False)
print('Fin.')

Fin.
